In [4]:
import yaml
import json
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

from rules_engine import rules_engine
import utils

In [14]:
with open('config.yaml') as f:
	# use safe_load instead load
	data_map = yaml.safe_load(f)
file_length = data_map['file_length']['value']


100

In [5]:
use_cols = ['name', 'metDivName', 'countyFips', 'geoIdMsa', 'metDivFp', 'smallCounty', 'tracts']
cbsa_cols = ['name', 'metDivName', 'state', 'countyFips', 'county', 'tracts','geoIdMsa', 'metDivFp', 'smallCounty', 
			 'stateCode', 'tractDecimal']
cbsas = pd.read_csv('../dependencies/tract_to_cbsa_2015.txt', usecols=use_cols, delimiter='|', 
					header=None, names=cbsa_cols, dtype=str) #load tract to CBSA data from platform file
cbsas["tractFips"] = cbsas.countyFips + cbsas.tracts
counties = list(cbsas.countyFips)
tracts = list(cbsas.tractFips)

#load schemas for LAR and transmittal sheet
#schemas contain valid enumerations, including NA values, for each field in the dataset
lar_schema_df = pd.DataFrame(json.load(open("../schemas/lar_schema.json", "r")))
ts_schema_df = pd.DataFrame(json.load(open("../schemas/ts_schema.json", "r")))
!pwd

/Users/roellk/Desktop/HMDA/hmda-test-files/python


In [6]:
path = "../edits_files/validity/"
file = "v628_2.txt"
#lar validator checks a dataframe and returns a JSON with generate_error_files
checker = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, counties=counties) 
#load data to checker
ts_df, lar_df = utils.read_data_file(path=path, data_file=file)
checker.load_data_frames(ts_df, lar_df)
for func in dir(checker):
    if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
        #print("applying:", func)
        getattr(checker, func)()
res_df = pd.DataFrame(checker.results)
res_df[(res_df.status=="failed")&(res_df.edit_name==file[:-4])]

,edit_name,fail_count,fields,row_ids,row_type,status
54,v628_2,100.0,app ethnicities 2-4,[G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...,LAR,failed


In [15]:
#get all edit test files in single list
val_path = "../edits_files/validity/"
syn_path = "../edits_files/syntax/"
val_files = [f for f in listdir(val_path) if isfile(join(val_path, f)) and f[0]=="v"] #get list of only files
syn_files = [f for f in listdir(syn_path) if isfile(join(syn_path, f)) and f[0]=="s"]

In [21]:
#check validity test files for error rate
for file in val_files:
    print(file)
    #lar validator checks a dataframe and returns a JSON with generate_error_files
    checker = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, counties=counties)
    #load data to checker
    ts_df, lar_df = utils.read_data_file(path=val_path, data_file=file)
    checker.load_data_frames(ts_df, lar_df)
    for func in dir(checker):
        if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
            getattr(checker, func)()
    res_df = pd.DataFrame(checker.results)
    res_df = res_df[(res_df.edit_name==file[:-4])] #set res_df for only the edit in the file name
    if res_df.row_type.iloc[0]=="TS" and res_df.status.iloc[0]=="failed":
        print("file is good")
        
    if res_df.row_type[(res_df.edit_name==file[:-4])].iloc[0] == "LAR" and \
       res_df.fail_count[(res_df.edit_name==file[:-4])].iloc[0] != file_length:
        print("WARNING BOOOOOP")
    elif res_df.row_type[(res_df.edit_name==file[:-4])].iloc[0] == "LAR" and fail_count == file_length:
        print("file is good")
    print()

v600.txt
  edit_name  fail_count fields  \
6      v600       100.0    LEI   

                                             row_ids row_type  status  
6  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v601_1.txt
  edit_name     fields row_type  status
7    v601_1  inst_name       TS  failed
happiness

v601_2.txt
  edit_name        fields row_type  status
8    v601_2  contact_name       TS  failed
happiness

v601_3.txt
  edit_name         fields row_type  status
9    v601_3  contact_email       TS  failed
happiness

v601_4.txt
   edit_name                  fields row_type  status
10    v601_4  contact_street_address       TS  failed
happiness

v602.txt
   edit_name            fields row_type  status
12      v602  calendar_quarter       TS  failed
happiness

v603.txt
   edit_name       fields row_type  status
13      v603  contact_tel       TS  failed
happiness

v604.txt
   edit_name        fields row_type  status
14      v604  office_state       TS  fa

   edit_name  fail_count    fields  \
46    v622_3       100.0  zip_code   

                                              row_ids row_type  status  
46  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v623.txt
   edit_name  fail_count fields  \
47      v623       100.0  state   

                                              row_ids row_type  status  
47  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v624.txt
   edit_name  fail_count    fields  \
48      v624       100.0  zip_code   

                                              row_ids row_type  status  
48  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v625_1.txt
   edit_name  fail_count fields  \
49    v625_1       100.0  tract   

                                              row_ids row_type  status  
49  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v625_2.txt
   edit_name  fail_count

   edit_name  fail_count                fields  \
82    v636_3       100.0  Applicant Race Basis   

                                              row_ids row_type  status  
82  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v637_1.txt
   edit_name  fail_count                fields  \
83    v637_1       100.0  Applicant Race Basis   

                                              row_ids row_type  status  
83  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v637_2.txt
   edit_name  fail_count            fields  \
84    v637_2       100.0  Applicant Race 1   

                                              row_ids row_type  status  
84  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v638_1.txt
   edit_name  fail_count               fields  \
85    v638_1       100.0  Co-Applicant Race 1   

                                              row_ids row_type  status  
85  [G3DA5TSXYTQXC12

    edit_name  fail_count         fields  \
113    v651_2       100.0  Applicant Age   

                                               row_ids row_type  status  
113  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v652_1.txt
    edit_name  fail_count            fields  \
114    v652_1       100.0  Co-Applicant Age   

                                               row_ids row_type  status  
114  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v652_2.txt
    edit_name  fail_count            fields  \
115    v652_2       100.0  Co-Applicant Age   

                                               row_ids row_type  status  
115  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v654_1.txt
    edit_name  fail_count  fields  \
116    v654_1       100.0  Income   

                                               row_ids row_type  status  
116  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432..

    edit_name  fail_count              fields  \
145    v669_3       100.0  Denial Reasons 1-4   

                                               row_ids row_type  status  
145  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v669_4.txt
    edit_name  fail_count              fields  \
146    v669_4       100.0  Denial Reasons 1-4   

                                               row_ids row_type  status  
146  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v670_1.txt
    edit_name  fail_count           fields  \
147    v670_1       100.0  Denial Reason 1   

                                               row_ids row_type  status  
147  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v670_2.txt
    edit_name  fail_count           fields  \
148    v670_2       100.0  Denial Reason 1   

                                               row_ids row_type  status  
148  [G3DA5TSXYTQXC12MS

    edit_name  fail_count          fields  \
176    v676_5       100.0  Lender Credits   

                                               row_ids row_type  status  
176  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v677_1.txt
    edit_name  fail_count         fields  \
177    v677_1       100.0  Interest Rate   

                                               row_ids row_type  status  
177  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v677_2.txt
    edit_name  fail_count         fields  \
178    v677_2       100.0  Interest Rate   

                                               row_ids row_type  status  
178  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v678_1.txt
    edit_name  fail_count           fields  \
179    v678_1       100.0  Prepayment Term   

                                               row_ids row_type  status  
179  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7

    edit_name  fail_count            fields  \
207    v692_1       100.0  Affordable Units   

                                               row_ids row_type  status  
207  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v692_2.txt
    edit_name  fail_count            fields  \
208    v692_2       100.0  Affordable Units   

                                               row_ids row_type  status  
208  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v692_3.txt
    edit_name  fail_count            fields  \
209    v692_3       100.0  Affordable Units   

                                               row_ids row_type  status  
209  [G3DA5TSXYTQXC12MSC6A6XM1LCODVBA2HMYGMD7Q2H432...      LAR  failed  
file is good

v694_1.txt
    edit_name  fail_count             fields  \
212    v694_1       100.0  initially_payable   

                                               row_ids row_type  status  
212  [G3DA5TSXYTQXC12MSC6

In [16]:
#check syntax test files for error rate
for file in syn_files:
    print(file)
    #lar validator checks a dataframe and returns a JSON with generate_error_files
    checker = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, counties=counties)
    #load data to checker
    ts_df, lar_df = utils.read_data_file(path=syn_path, data_file=file)
    checker.load_data_frames(ts_df, lar_df)
    for func in dir(checker):
        if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
            getattr(checker, func)()
    res_df = pd.DataFrame(checker.results)
    res_df = res_df[(res_df.edit_name==file[:-4])] #set res_df for only the edit in the file name
    
    if res_df.row_type.iloc[0]=="TS" and res_df.status.iloc[0]=="failed":
        print("file is good")
    if res_df.row_type[(res_df.edit_name==file[:-4])].iloc[0] == "LAR":
        fail_count = res_df.fail_count[(res_df.edit_name==file[:-4])].iloc[0]
    
    if row_type == "LAR" and int(fail_count) != file_length:
        print("WARNING BOOOOOP\n", "*"*100)
    else:
        print("file is good")
    print()

s300_1.txt
  edit_name     fields row_type  status
0    s300_1  record_id       TS  failed
file is good

s300_2.txt
file is good

s301.txt
  edit_name fields row_type  status
2      s301    LEI       TS  failed
file is good

s302.txt
  edit_name         fields row_type  status
3      s302  calendar_year       TS  failed
file is good

s304.txt
file is good

s305.txt
file is good

